In [1]:
from selenium.webdriver import Chrome

In [2]:
web = Chrome(executable_path='E:\Google\Chrome\Application\chromedriver.exe')

In [3]:
web.get('https://shimo.im/login?from=home')

In [4]:
user = web.find_element_by_xpath('//*[@name="mobileOrEmail"]')
user

<selenium.webdriver.remote.webelement.WebElement (session="e9f9d0c63298373d80e91480cb1fb21b", element="0.8061558003732576-1")>

In [6]:
user = web.find_element_by_xpath('//*[@name="mobileOrEmail"]')
user.send_keys('63888590@qq.com')
password = web.find_element_by_xpath('//*[@name="password"]')
password.send_keys('123456789')
logon = web.find_element_by_xpath('//*[@class="sm-button submit sc-1n784rm-0 bcuuIb"]')
logon.click()

<selenium.webdriver.remote.webelement.WebElement (session="e9f9d0c63298373d80e91480cb1fb21b", element="0.8061558003732576-3")>

In [13]:
password = web.find_element_by_xpath('//*[@name="password"]')
password

<selenium.webdriver.remote.webelement.WebElement (session="771632d835b0d4e10be03c304b59d7d8", element="0.9584558218164494-2")>

In [14]:
password.send_keys('123456789')

In [16]:
logon = web.find_element_by_xpath('//*[@class="sm-button submit sc-1n784rm-0 bcuuIb"]')
logon

<selenium.webdriver.remote.webelement.WebElement (session="771632d835b0d4e10be03c304b59d7d8", element="0.9584558218164494-3")>

In [17]:
logon.click()

In [ ]:
##web.execute_script("document.getElementsByClassName('geetest_refresh')[0].click()")

In [18]:
import requests
response = requests.get('https://api.geetest.com/get.php?is_next=true&type=click&gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=cd86bea66c083fa774af75a03950b8f0&lang=zh-cn&https=true&protocol=https%3A%2F%2F&offline=false&product=embed&api_server=api.geetest.com&isPC=true&width=100%25&callback=geetest_1591004891395').text
response

'geetest_1591004891395({"user_error": "\\u7f51\\u7edc\\u4e0d\\u7ed9\\u529b", "error_code": "error_02", "error": "old challenge", "status": "error"})'

In [32]:
def get_challenge(web):
    selector = HTML(web.page_source)
    data = selector.xpath('/html/head/script[7]/@src')
    challenge = re.findall('&challenge=(.*?)&',str(data),re.S)
    return challenge

In [40]:
from lxml.etree import HTML
import re
selector = HTML(web.page_source)
data = selector.xpath('/html/head/script[7]/@src')
challenge = re.findall('&challenge=(.*?)&',str(data),re.S)
challenge

['4dbf6c274453aa1b8f24714e655bc81b']

In [35]:
def get_gt(web):
    ele = web.find_element_by_xpath("//*[@charset='UTF-8'][2]")
    gt = ele.get_attribute('src').split('=')[1].split('&')[0]
    return gt
gt = get_gt(web)
gt

'be6b1ead17b7bd4e64d9b27e7051004a'

In [41]:
import time
round(time.time()*1000)
response = requests.get('https://api.geetest.com/get.php?is_next=true&type=click&gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge={}&lang=zh-cn&https=true&protocol=https%3A%2F%2F&offline=false&product=embed&api_server=api.geetest.com&isPC=true&width=100%25&callback=geetest_{}'.format('931d1875d737e98b3fdf48a49986d2ee',round(time.time()*1000))).text
response

'geetest_1591015898708({"user_error": "\\u7f51\\u7edc\\u4e0d\\u7ed9\\u529b", "error_code": "error_02", "error": "old challenge", "status": "error"})'

In [65]:
from retrying import retry

from wget import download


# 传入参数为selenium的浏览器（状态为第一次进入页面），需要触发一次显示验证码并返回对应的图片地址


def first_click(web):
    logon = web.find_element_by_xpath('//*[@class="sm-button submit sc-1n784rm-0 bcuuIb"]')
    logon.click()
    time.sleep(3)
    ele = web.find_elements_by_xpath('//*[@class="geetest_item_img"]')
    for i in ele:
        pic_url = i.get_attribute('src').split('?')[0]
    return pic_url

def retry_error(web):
    web.execute_script("document.getElementsByClassName('geetest_panel_error_content')[0].click()")
    time.sleep(1)
    ele = web.find_elements_by_xpath('//*[@class="geetest_item_img"]')
    for i in ele:
        pic_url = i.get_attribute('src').split('?')[0]
    return pic_url


# 传入参数为selenium的浏览器（状态为触发5次之后的重试），需要触发一次显示验证码并返回对应的图片地址


def retry_clikc(web):
    web.execute_script(
        "document.getElementsByClassName('geetest_refresh')[0].click()")
    time.sleep(2)
#     return retry_clikc_child(web)
    ele = web.find_elements_by_xpath('//*[@class="geetest_item_img"]')
    for i in ele:
        pic_url = i.get_attribute('src').split('?')[0]
    return pic_url

error_img_list = []  # 错误列表
# picurlQ = []  # 下载图片任务队列
picSet = set()  # 记录所有下载过的图片的集合
ar_num = 0

def run_scrape():
    task_num = 100000  # 爬多少个验证码停止（包括重复的）
    cnt = 0  # 用于记录一部分总数
    cnt_c = 0  # 用于记录重试次数
    du_cnt = 0  # 用于记录重复的次数
    # thread_num = 5  # 下载的线程数
    # down_thread = []  # 线程list
    global web
    web = Chrome(executable_path='E:\Google\Chrome\Application\chromedriver.exe')  # 使用Chrome打开，WebDrive放置在bin目录下因此不需要指定路径
    web.get("https://shimo.im/login?from=home")
    user = web.find_element_by_xpath('//*[@name="mobileOrEmail"]')
    user.send_keys('63888590@qq.com')
    password = web.find_element_by_xpath('//*[@name="password"]')
    password.send_keys('123456789')
#         logon = web.find_element_by_xpath('//*[@class="sm-button submit sc-1n784rm-0 bcuuIb"]')
#         logon.click()
    t_url = first_click(web)  # 初始页面第一次点击
    # picurlQ.append(t_url)
    picSet.add(t_url)
    fd.write(t_url + '\n')
    gt = get_gt(web)  # 获取gt值（从头到位不会变）
    with open("gt.txt", 'w') as td:
        td.write(gt)
    challenge = get_challenge(web)  # 获取第一个challenge值

    # 开启用于下载图片的线程
    # for i in range(thread_num):
    #     down_thread.append(myThread(i))
    #     down_thread[i].start()

    # 主爬取循环
    while cnt + cnt_c + ar_num < task_num:
        # 每5次需要点击重试
        if cnt % 5 == 0 and cnt != 0:
            cnt_c += 1
            t_url = retry_error(web)  # 点击重试
            # 如果这个url已经在里面，就跳过
#             challenge = get_challenge(web)  # 获取challenge
            if t_url in picSet:
                du_cnt += 1
                print("检测到重复链接，当前总计重复{}张，本次任务重复率 {:.2f}%".format(
                    du_cnt, (du_cnt / (cnt + cnt_c)) * 100))

            else:
                picSet.add(t_url)
                fd.write(t_url + '\n')
                # picurlQ.append(t_url)
                print("已爬取 {} 个验证码链接   完成 {:.2f}%".format(cnt + cnt_c, ((cnt + cnt_c + ar_num) / task_num) * 100))
        url = "https://api.geetest.com/refresh.php?gt={}&challenge={}&lang=zh-cn&type=click&callback=geetest_{}".format(
            gt, challenge, int(round(time.time() * 1000)+3000))
        print(url)
        data = requests.get(url).text
        print(data)
        t_url = "https://static.geetest.com" + re.search(r"pic\": \"(.+?)\"", str(data)).group(1)
        t_url = retry_clikc(web)
        cnt += 1
        if t_url in picSet:
            du_cnt += 1
            print("检测到重复链接，当前总计重复{}张，本次任务重复率 {:.2f}%".format(
                du_cnt, (du_cnt / (cnt + cnt_c)) * 100))
            continue
        picSet.add(t_url)
        fd.write(t_url + '\n')
#     except Exception as e:
#         web.close()
#         run_scrape()
        # picurlQ.append(t_url)
        # finish = True
        # for i in range(thread_num):
        #     down_thread[i].join()
    web.close()


# 打开用于记录URL的文件
fd = open("image_url.txt", 'r')

# 把所有文件中已经读到的添加到set中
line = fd.readline()
while line:
    picSet.add(line)
    line = fd.readline()
ar_num = len(picSet)
fd.close()

fd = open("image_url.txt", 'a')
# 开始运行爬虫]
run_scrape()

检测到重复链接，当前总计重复1张，本次任务重复率 16.67%
检测到重复链接，当前总计重复2张，本次任务重复率 25.00%
检测到重复链接，当前总计重复3张，本次任务重复率 33.33%
检测到重复链接，当前总计重复4张，本次任务重复率 40.00%
检测到重复链接，当前总计重复5张，本次任务重复率 45.45%
已爬取 12 个验证码链接   完成 0.34%


KeyboardInterrupt: 

In [59]:
1591017382899-1591017379441

3458

In [75]:
from selenium.webdriver import Chrome
import time
import requests
import threading
from lxml.etree import HTML
import re
from retrying import retry

from wget import download


# 传入参数为selenium的浏览器（状态为第一次进入页面），需要触发一次显示验证码并返回对应的图片地址


def first_click(web):
    logon = web.find_element_by_xpath('//*[@class="sm-button submit sc-1n784rm-0 bcuuIb"]')
    logon.click()
    time.sleep(3)
    ele = web.find_elements_by_xpath('//*[@class="geetest_item_img"]')
    for i in ele:
        pic_url = i.get_attribute('src').split('?')[0]
    return pic_url


@retry(stop_max_attempt_number=10, wait_random_min=500, wait_random_max=1000)
def retry_clikc_child(web):
    web.execute_script("document.getElementsByClassName('geetest_panel_error_content')[0].click()")
    time.sleep(1)
    ele = web.find_elements_by_xpath('//*[@class="geetest_item_img"]')
    for i in ele:
        pic_url = i.get_attribute('src').split('?')[0]
        challenge = i.get_attribute('src').split('=')[-1]
    return pic_url,challenge


# 传入参数为selenium的浏览器（状态为触发5次之后的重试），需要触发一次显示验证码并返回对应的图片地址


def retry_clikc(web):
    web.execute_script("document.getElementsByClassName('geetest_refresh')[0].click()")
    time.sleep(2)
    return retry_clikc_child(web)


# 传入参数为selenium的浏览器，需要在页面中找到gt并返回


def get_gt(web):
    ele = web.find_element_by_xpath("//*[@charset='UTF-8'][2]")
    gt = ele.get_attribute('src').split('=')[1].split('&')[0]
    return gt


# 传入参数为selenium的浏览器（状态为显示第一张验证码完后），需要在页面中找到challenge并返回


def get_challenge(web):
    selector = HTML(web.page_source)
    data = selector.xpath('/html/head/script[7]/@src')
    challenge = re.findall('&challenge=(.*?)&',str(data),re.S)
    return challenge


class myThread(threading.Thread):
    def __init__(self, threadID):
        threading.Thread.__init__(self)
        self.threadID = threadID

    def run(self):
        print("开始线程：" + self.name)
        download_image(self.threadID)
        print("退出线程：" + self.name)


def download_image(threadID):
    while not finish:
        if (len(picurlQ) == 0):
            time.sleep(1)
            continue
        cur = picurlQ.pop()
        # print("\nthread " + str(threadID) + " 开始下载 " + cur[0].split('/')[-1])
        img_path = 'imgs/{}'.format(cur.split('/')[-1])
        try:
            download(cur, img_path)
        except Exception as e:
            print("\n" + str(e))
            error_img_list.append([cur, str(e)])


error_img_list = []  # 错误列表
# picurlQ = []  # 下载图片任务队列
picSet = set()  # 记录所有下载过的图片的集合
ar_num = 0


# finish = False  # 完成标示


def run_scrape():
    task_num = 100000  # 爬多少个验证码停止（包括重复的）
    cnt = 0  # 用于记录一部分总数
    cnt_c = 0  # 用于记录重试次数
    du_cnt = 0  # 用于记录重复的次数
    # thread_num = 5  # 下载的线程数
    # down_thread = []  # 线程list
    try:
        global web
        web = Chrome(executable_path='E:\Google\Chrome\Application\chromedriver.exe')  # 使用Chrome打开，WebDrive放置在bin目录下因此不需要指定路径
        web.get("https://shimo.im/login?from=home")
        user = web.find_element_by_xpath('//*[@name="mobileOrEmail"]')
        user.send_keys('63888590@qq.com')
        password = web.find_element_by_xpath('//*[@name="password"]')
        password.send_keys('123456789')
        t_url = first_click(web)  # 初始页面第一次点击
        # picurlQ.append(t_url)
        picSet.add(t_url)
        fd.write(t_url + '\n')
        gt = get_gt(web)  # 获取gt值（从头到位不会变）
        with open("gt.txt", 'w') as td:
            td.write(gt)
        challenge = get_challenge(web)[0]  # 获取第一个challenge值

        # 开启用于下载图片的线程
        # for i in range(thread_num):
        #     down_thread.append(myThread(i))
        #     down_thread[i].start()

        # 主爬取循环
        while cnt + cnt_c + ar_num < task_num:
            # 每5次需要点击重试
            if cnt % 5 == 0 and cnt != 0:
                cnt_c += 1
                t_url,challenge= retry_clikc(web)  # 点击重试
                # 如果这个url已经在里面，就跳过
                if t_url in picSet:
                    du_cnt += 1
                    print("检测到重复链接，当前总计重复{}张，本次任务重复率 {:.2f}%".format(
                        du_cnt, (du_cnt / (cnt + cnt_c)) * 100))

                else:
                    picSet.add(t_url)
                    fd.write(t_url + '\n')
                    # picurlQ.append(t_url)
                    print("已爬取 {} 个验证码链接   完成 {:.2f}%".format(cnt + cnt_c, ((cnt + cnt_c + ar_num) / task_num) * 100))
            url = "https://api.geetest.com/refresh.php?gt={}&challenge={}&lang=zh-cn&type=click&callback=geetest_{}".format(
                gt, challenge, int(round(time.time() * 1000)))
            data = requests.get(url).text
            t_url = "https://static.geetest.com" + re.search(r"pic\": \"(.+?)\"", data).group(1)
            cnt += 1
            if t_url in picSet:
                du_cnt += 1
                print("检测到重复链接，当前总计重复{}张，本次任务重复率 {:.2f}%".format(
                    du_cnt, (du_cnt / (cnt + cnt_c)) * 100))
                continue
            picSet.add(t_url)
            fd.write(t_url + '\n')
    except Exception as e:
        web.close()
        run_scrape()
        # picurlQ.append(t_url)
        # finish = True
        # for i in range(thread_num):
        #     down_thread[i].join()
    web.close()


# 打开用于记录URL的文件
fd = open("image_url.txt", 'r')

# 把所有文件中已经读到的添加到set中
line = fd.readline()
while line:
    picSet.add(line)
    line = fd.readline()
ar_num = len(picSet)
fd.close()

fd = open("image_url.txt", 'a')
# 开始运行爬虫]
run_scrape()
# flag = 0
# while True:
#     try:
#         if flag==1:
#             flag=0
#         run_scrape()
#     except Exception as e:
#         fd.close()
#         flag = 1

# 关闭文件
fd.close()


https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=ba5a6ef8a1a462ff02e379f508ae35f1&lang=zh-cn&type=click&callback=geetest_1591022894780
geetest_1591022894780({"status": "success", "data": {"sign": "", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "spec": "1*1", "num": 0, "pic_type": "word", "pic": "/nerualpic/word_l1_zh_2020.03.16/owl/5cc1606e75f4116a4c0d4104e93a0131.jpg"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=ba5a6ef8a1a462ff02e379f508ae35f1&lang=zh-cn&type=click&callback=geetest_1591022894988
geetest_1591022894988({"status": "success", "data": {"image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "num": 0, "spec": "1*1", "pic_type": "word", "sign": "", "pic": "/nerualpic/word_l1_zh_2020.03.16/abstract/51139c81b6ad126836ab24068da054cb.jpg"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=ba5a6ef8a1a462ff02e379f508ae35f1&lang=zh-cn&type=

geetest_1591022907565({"status": "success", "data": {"spec": "1*1", "pic": "/nerualpic/word_l1_zh_2020.03.16/cubism6/4555d4c6e9b15e3283ffc7a6ac1cef79.jpg", "sign": "", "num": 0, "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic_type": "word"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=cc7c869cf9e17582b9fe3cd193dc5a41&lang=zh-cn&type=click&callback=geetest_1591022907769
geetest_1591022907769({"status": "success", "data": {"pic": "/nerualpic/word_l1_zh_2020.03.16/abstract/0d7600de3038f705284895bce6b33abc.jpg", "pic_type": "word", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "num": 0, "spec": "1*1", "sign": ""}})
e7c041de111d3f895b952384bc2a90df
已爬取 24 个验证码链接   完成 0.40%
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=e7c041de111d3f895b952384bc2a90df&lang=zh-cn&type=click&callback=geetest_1591022911032
geetest_1591022911032({"status": "success", "data": {"pic": "/nerualp

geetest_1591022923727({"status": "success", "data": {"pic": "/nerualpic/word_l1_zh_2020.03.16/girl2/443c5d8ea81b83cc19536c5b02fbbc5b.jpg", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic_type": "word", "spec": "1*1", "sign": "", "num": 0}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=1f579a57c34ffc1a6febe9aed3775ee8&lang=zh-cn&type=click&callback=geetest_1591022923936
geetest_1591022923936({"status": "success", "data": {"sign": "", "spec": "1*1", "pic": "/nerualpic/word_l1_zh_2020.03.16/harley1/2062fdd7b00360a6a6c9b4ea10414634.jpg", "pic_type": "word", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "num": 0}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=1f579a57c34ffc1a6febe9aed3775ee8&lang=zh-cn&type=click&callback=geetest_1591022924141
geetest_1591022924141({"status": "success", "data": {"pic": "/nerualpic/word_l1_zh_2020.03.16/abstract/d0800e91bbd96ff80e70b3e195b

geetest_1591022939698({"status": "success", "data": {"pic": "/nerualpic/word_l1_zh_2020.03.16/abstract/458f44f4569d1b6e3f31635b73dc806d.jpg", "spec": "1*1", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic_type": "word", "num": 0, "sign": ""}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=5ca5888b73e6ef2dc71e1701cc17c236&lang=zh-cn&type=click&callback=geetest_1591022939882
geetest_1591022939882({"status": "success", "data": {"spec": "1*1", "num": 0, "sign": "", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic_type": "word", "pic": "/nerualpic/word_l1_zh_2020.03.16/owl/b6431ab9c19a83f76f8f6ffdb3bfd3dc.jpg"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=5ca5888b73e6ef2dc71e1701cc17c236&lang=zh-cn&type=click&callback=geetest_1591022940074
geetest_1591022940074({"status": "success", "data": {"num": 0, "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], 

b16c35142d7e6bebacf75efa030f2e68
已爬取 90 个验证码链接   完成 0.46%
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=b16c35142d7e6bebacf75efa030f2e68&lang=zh-cn&type=click&callback=geetest_1591022955709
geetest_1591022955709({"status": "success", "data": {"sign": "", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "spec": "1*1", "num": 0, "pic_type": "word", "pic": "/nerualpic/word_l1_zh_2020.03.16/owl/f30913ad41ee9e6c9e3ae939ae5e077a.jpg"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=b16c35142d7e6bebacf75efa030f2e68&lang=zh-cn&type=click&callback=geetest_1591022955917
geetest_1591022955917({"status": "success", "data": {"spec": "1*1", "pic": "/nerualpic/word_l1_zh_2020.03.16/cubism6/08c94733896aac36fda4857287f7e738.jpg", "sign": "", "num": 0, "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic_type": "word"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&ch

geetest_1591022968596({"status": "success", "data": {"sign": "", "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "spec": "1*1", "num": 0, "pic_type": "word", "pic": "/nerualpic/word_l1_zh_2020.03.16/harley1/d0915bf0ac9df6894bef6012db738c1c.jpg"}})
https://api.geetest.com/refresh.php?gt=be6b1ead17b7bd4e64d9b27e7051004a&challenge=57f80d81f3c80068894a30117c38d63b&lang=zh-cn&type=click&callback=geetest_1591022968797
geetest_1591022968797({"status": "success", "data": {"num": 0, "image_servers": ["static.geetest.com/", "dn-staticdown.qbox.me/"], "pic": "/nerualpic/word_l1_zh_2020.03.16/cubism6/4d784cbcc974c95e272285b6943ae5d1.jpg", "spec": "1*1", "sign": "", "pic_type": "word"}})


KeyboardInterrupt: 